In [2]:
%load_ext autoreload
%autoreload 2

import sys
import os 
import pandas as pd
import numpy as np

import soepy

from dev_library import plot_basics_choices
from dev_library import plot_basics_wages
from dev_library import *
import pandas as pd
pd.set_option('display.max_rows', 500)
sys.path.insert(0, "/home/peisenha/structExpectations/development/submodules/soepy/doc/tutorial")
os.chdir("/home/peisenha/structExpectations/development/submodules/soepy/doc/tutorial")


In [5]:
params = pd.read_pickle("model_params.pkl")
params

value  upper  lower  fixed
category          name                                                 
const_wage_eq     gamma_0s1               1.939968  3.700  1.380      0
                  gamma_0s2               2.053860  3.700  1.380      0
                  gamma_0s3               2.189363  3.700  1.380      0
exp_returns       gamma_1s1               0.154747  0.500  0.001      0
                  gamma_1s2               0.202052  0.500  0.001      0
                  gamma_1s3               0.287655  0.500  0.001      0
exp_accm          g_s1                    0.118576  0.500  0.001      0
                  g_s2                    0.273697  0.500  0.001      0
                  g_s3                    0.251750  0.500  0.001      0
exp_accm_expected g_bar_s1                0.570000  1.000  0.001      1
                  g_bar_s2                0.535000  1.000  0.001      1
                  g_bar_s3                0.625000  1.000  0.001      1
exp_deprec        delta_s1                0.020000  0.200  0.001      0
                  delta_s2                0.020000  0.200  0.001      0
                  delta_s3                0.020000  0.200  0.001      0
hetrg_unobs       theta_p1               -0.624943 -0.100 -1.000      0
                  theta_f1               -0.940325 -0.100 -1.000      0
shares            share_1                 0.660619  0.900  0.001      0
sd_wage_shock     sigma_1                 0.334045  1.000  0.001      0
                  sigma_2                 0.444563  1.000  0.001      0
disutil_work      no_kids_f_educ_low      0.538150  0.200  0.001      0
                  no_kids_f_educ_middle   0.538150  0.200  0.001      0
                  no_kids_f_educ_high     0.538150  0.200  0.001      0
                  yes_kids_f_educ_low     1.448252  0.200  0.001      0
                  yes_kids_f_educ_middle  1.448252  0.200  0.001      0
                  yes_kids_f_educ_high    1.448252  0.200  0.001      0
                  child_02_f              0.088035  0.200  0.001      0
                  child_35_f              0.001163  0.200  0.001      0
                  child_610_f             0.027891  0.200  0.001      0
                  no_kids_p_educ_low     -0.443981 -0.001 -1.500      0
                  no_kids_p_educ_middle  -0.443981 -0.001 -1.500      0
                  no_kids_p_educ_high    -0.443981 -0.001 -1.500      0
                  yes_kids_p_educ_low    -0.497672 -0.001 -1.500      0
                  yes_kids_p_educ_middle -0.497672 -0.001 -1.500      0
                  yes_kids_p_educ_high   -0.497672 -0.001 -1.500      0
                  child_02_p             -0.270179 -0.001 -1.500      0
                  child_35_p             -0.341858 -0.001 -1.500      0
                  child_610_p            -0.410474 -0.001 -1.500      0